In [1]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor
from ipyregulus.tree.has_tree import *
from ipyregulus.alg.view import *

In [2]:
data = io.load('data/gauss4')
# data = io.load('data/simulations')

In [3]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

#### reduce

In [ ]:
tw1 = TreeWidget(data.tree)

In [ ]:
tw2 = TreeWidget(tw1)

In [ ]:
af = AttrFilter(attr='span', func=lambda x, v: v<=x)

In [ ]:
display(af)

In [ ]:
m, dest = reduce_tree(tw1, af, tw2)

In [ ]:
v2 = TreeView(tw2)

In [ ]:
display(v2)

In [ ]:
v3, m3, f, p3, d3 = show_reduce(tw1, af)

In [ ]:
v3.attr = 'fitness'

In [ ]:
sr = show_reduce(tw1, panel='Reduce example')

### Initial tree

In [4]:
tw = TreeWidget(data.tree)

In [10]:
v1, m1, f1, p1 = show_tree(tw, panel='Initial Tree')

In [11]:
v2, m2, f2, p2 = show_tree(tw, attr='fitness', panel='fitness')

In [12]:
f2.func = lambda x, v: v > x

In [ ]:
v2.attr = 'parent_fitness'

In [13]:
f2.attr = 'span'
f2.func = lambda x, v: v <=x

### Reduce tree

In [ ]:
v3, r3, p3 = reduce(data, attr='span', title='reduce')

In [ ]:
v3.attr = 'parent_fitness'

### fitness

#### test

In [ ]:
tw2 = TreeWidget(data.tree)

In [ ]:
f4 = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [ ]:
def do_reduce(f):
    def _reduce(*args):
        tw2.set(reduce_tree(tw.tree, filter=f))
    return _reduce

In [ ]:
v4 = TreeView(tree=tw2, attr='span') 

In [ ]:
m4 = Monitor(f4, func=do_reduce(f4))

In [ ]:
p4 = SidePanel(title='Test')
with p4:
    display(v4, f4)

In [ ]:
fitness = view(tw2, attr='fitness', title='fitness')

In [ ]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

### Data widget

In [ ]:
dw = DataWidget(data=data)

In [ ]:
details = DetailsView(data=dw)

In [ ]:
p7 = SidePanel(title='details')
with p7:
    display(details)

In [ ]:
widgets.dlink((fitness[0], 'details'), (details, 'show'))

In [ ]:
@debug.capture()
def test(nv, av):
    print('test', nv, av)
    return nv < av

In [ ]:
a = AttrFilter(attr='span', func=test)

In [ ]:
display(a)

In [ ]:
a(tw.tree, tw.tree.root)

In [ ]:
@debug.capture()
def test2(*args, **kwargs):
    print('test2', *args, **kwargs)

In [ ]:
m = Monitor(a, func=test2)

In [ ]:
a.observe(test2, names='changed')

In [ ]:
a.changed

In [ ]:
a.ui

In [ ]:
a.ui.__dict__

In [ ]:
a.ui.observe(test2, names='value')

In [ ]:
if None in (8, None):
    print('yes')